In [ ]:
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
import scipy.io
import os.path
import sys
%matplotlib inline

In [ ]:
class MyInfo:
    # Give names of class members.
    def __repr__(self):
        return self.__class__.__name__ + '(' + str(list(self.__dict__.keys())) + ')'
    def __str__(self):
        return self.__class__.__name__ + '(' + str(list(self.__dict__.keys())) + ')'

In [ ]:
def get_color_list(cmap, N=None, use_index="auto", reverse=False):
    import numpy as np
    from matplotlib.colors import LinearSegmentedColormap, ListedColormap
    import matplotlib
    if isinstance(cmap, str):
        if use_index == "auto":
            if cmap in ['Pastel1', 'Pastel2', 'Paired', 'Accent',
                        'Dark2', 'Set1', 'Set2', 'Set3',
                        'tab10', 'tab20', 'tab20b', 'tab20c']:
                use_index=True
            else:
                use_index=False
        try:
            # version 3.5 and later
            cmap = matplotlib.colormaps.get_cmap(cmap)
        except AttributeError:
            cmap = matplotlib.pyplot.get_cmap(cmap)
    if N is None:
        N = cmap.N
    if use_index=="auto":
        if cmap.N > 100:
            use_index=False
        elif isinstance(cmap, LinearSegmentedColormap):
            use_index=False
        elif isinstance(cmap, ListedColormap):
            use_index=True
    if use_index:
        if reverse:
            ind = np.flip(np.arange(int(N))) % cmap.N
        else:
            print("debug")
            ind = np.arange(int(N)) % cmap.N
        return cmap(ind)
    else:
        if reverse:
            colors = cmap(np.flip(np.linspace(0,1,N)))
        else:
            colors = cmap(np.linspace(0,1,N))
        return colors
# https://stackoverflow.com/questions/30079590/use-matplotlib-color-map-for-color-cycle

In [ ]:
def out_path(filename):
    global outdir
    path = os.path.join(outdir, filename)
    return path

In [ ]:
def closest_index(arr, val):
    if val > arr.max():
        raise ValueError("not in range: {} > {}".format(val, arr.max()))
    elif val < arr.min():
        raise ValueError("not in range: {} < {}".format(val, arr.min()))
    index = np.argmin(abs(arr - val))  # TODO: use binary search or something else?
    return index

In [ ]:
nb_id = 1732778863
import time
try:
    print(nb_id)
except NameError:
    print(round(time.time()))

In [ ]:
filepaths = glob.glob("amp_series_03/*.mat")

In [ ]:
# filepaths

In [ ]:
outdir = "plot_FeGaB_amp_series_03"
filename_prefix = "FeGaB_amp_series_03"
title_prefix = "FeGaB, power series"

In [ ]:
gauss = 1e-4
GHz = 1e9

In [ ]:
# colors = {i: plt.get_cmap('tab20').colors[i] for i, val in enumerate(filepaths)}
# colors = {i: get_color_list('copper', N=len(filepaths)+3)[3:][i] for i, val in enumerate(filepaths)}
colors = {i: get_color_list('copper', N=len(filepaths))[i] for i, val in enumerate(filepaths)}

In [ ]:
if not os.path.isdir(outdir):
    os.mkdir(outdir)

In [ ]:
dat = {}
for i, filepath in enumerate(filepaths):
    dat[i] = scipy.io.loadmat(filepath, simplify_cells=True)

In [ ]:
bstat = np.array([d['bstat'] for (i, d) in dat.items()])
amp = np.array([d['amp'] for (i, d) in dat.items()])

In [ ]:
for i, (a, b) in enumerate(zip(amp, amp[1:])):
    print("{}\t{:g}\t{:.3f}\t{}".format(i+1, a/gauss, (a/b), dat[i]['folder']))

In [ ]:

fft_freq_Hz = dat[0]['fft_freq_Hz']
i0 = closest_index(fft_freq_Hz, 0.0)
i_max = closest_index(fft_freq_Hz, 5*GHz)

# Mz

In [ ]:
fig, ax = plt.subplots(
    constrained_layout=True,
    figsize=(16*0.7,9*0.7),
)
for i, d in dat.items():
    if i in (0, len(dat)-1):
        this_label = "$B_1 = ${:.1f} G".format(d['amp']/gauss)
    else:
        this_label = None
    ax.plot(
        d['fft_freq_Hz'][i0:i_max]/GHz,
        d['fftz_avg'][i0:i_max],
        label=this_label,
#         label="{:.1f} G".format(d['bstat']/gauss),
        color=colors[i],
    )
# ax.plot(fft_freq_Hz[i0:i_max]/GHz, fft_avg.z[i0:i_max]);
ax.set_title("$M_z$, " + title_prefix + ', nb_id = {}'.format(nb_id))
ax.set_xlabel('frequency [GHz]')
ax.set_ylabel('FFT mag. ($M_z$/$M_s$), linear scaling');
ax.axvline(1425e6/GHz, color='gray', linestyle='--', zorder=0, label='1425 MHz')
ax.axvline(1.5*1425e6/GHz, color='black', linestyle=':', zorder=0, label='{:.1f} MHz'.format(1.5*1425))
# ax.set_yscale('log')
ax.legend();

In [ ]:
fig.savefig(out_path(filename_prefix + "_overlay_fftz_linear.png"), bbox_inches='tight', facecolor="w", dpi=300);

In [ ]:
fig.canvas.draw()

In [ ]:
plt.close(fig); del(fig, ax)

In [ ]:
fig, ax = plt.subplots(
    constrained_layout=True,
    figsize=(16*0.7,9*0.7),
)
for i, d in dat.items():
    if i in (0, len(dat)-1):
        this_label = "$B_1 = ${:.1f} G".format(d['amp']/gauss)
    else:
        this_label = None
    ax.plot(
        d['fft_freq_Hz'][i0:i_max]/GHz,
        d['fftz_avg'][i0:i_max],
        label=this_label,
        color=colors[i],
    )
ax.set_title("$M_z$, " + title_prefix + ', nb_id = {}'.format(nb_id))
ax.set_xlabel('frequency [GHz]')
ax.set_ylabel('FFT mag. ($M_z$/$M_s$), log scaling');
ax.axvline(1425e6/GHz, color='gray', linestyle='--', zorder=0, label='1425 MHz')
ax.axvline(1.5*1425e6/GHz, color='black', linestyle=':', zorder=0, label='{:.1f} MHz'.format(1.5*1425))
ax.set_yscale('log')
ax.legend();

In [ ]:
fig.savefig(out_path(filename_prefix + "_overlay_fftz_log.png"), bbox_inches='tight', facecolor="w", dpi=300);

In [ ]:
fig.canvas.draw()

In [ ]:
plt.close(fig); del(fig, ax)

In [ ]:
Mz_harmonic0_list = []
Mz_harmonic1_list = []
Mz_harmonic0p5_list = []
for i, d in dat.items():
    Mz_harmonic0_list.append(d['fftz_avg'][closest_index(d['fft_freq_Hz'], 1425.0e6)])
    Mz_harmonic1_list.append(d['fftz_avg'][closest_index(d['fft_freq_Hz'], 2*1425.0e6)])
    Mz_harmonic0p5_list.append(d['fftz_avg'][closest_index(d['fft_freq_Hz'], 1.5*1425.0e6)])
Mz_harmonic0 = np.array(Mz_harmonic0_list)
Mz_harmonic1 = np.array(Mz_harmonic1_list)
Mz_harmonic0p5 = np.array(Mz_harmonic0p5_list)
Mz_harmonic_amp_ratio = Mz_harmonic1/Mz_harmonic0

In [ ]:
Mz_harmonic_amp_ratio

In [ ]:
color_list = list(colors.values())

In [ ]:
fig, ax = plt.subplots(
    constrained_layout=True,
    figsize=(16*0.7,9*0.7),
)
ax.plot(
    amp/gauss,
    Mz_harmonic0,
    '.-',
    label='1425 MHz'
)
ax.plot(
    amp/gauss,
    Mz_harmonic1,
    '.-',
    label='2850 MHz',
)
ax.plot(
    amp/gauss,
    Mz_harmonic0p5,
    '.-',
    label='{:.1f} MHz'.format(1425*1.5),
)
ax.set_title("$M_z$, " + title_prefix + ', nb_id = {}'.format(nb_id))
ax.set_xlabel('$B_1$ [G]')
ax.set_ylabel('FFT mag. ($M_z$/$M_s$)');
ax.axvline(7.54, color='gray', linestyle='--', zorder=0, label='7.54 G')
ax.set_yscale('log')
ax.legend();

In [ ]:
fig.savefig(out_path(filename_prefix + "_Mz_harmonic_0_1_vs_B1_log.png"), bbox_inches='tight', facecolor="w", dpi=300);

In [ ]:
fig.canvas.draw()

In [ ]:
plt.close(fig); del(fig, ax)

In [ ]:
fig, ax = plt.subplots(
    constrained_layout=True,
    figsize=(16*0.7,9*0.7),
)
ax.plot(
    amp/gauss,
    Mz_harmonic0,
    '.-',
    label='1425 MHz'
)
ax.plot(
    amp/gauss,
    Mz_harmonic1,
    '.-',
    label='2850 MHz',
)
ax.plot(
    amp/gauss,
    Mz_harmonic0p5,
    '.-',
    label='{:.1f} MHz'.format(1425*1.5),
)
ax.set_title("$M_z$, " + title_prefix + ', nb_id = {}'.format(nb_id))
ax.set_xlabel('$B_1$ [G]')
ax.set_ylabel('FFT mag. ($M_z$/$M_s$)');
ax.axvline(7.54, color='gray', linestyle='--', zorder=0, label='7.54 G')
ax.legend();

In [ ]:
fig.savefig(out_path(filename_prefix + "_Mz_harmonic_0_1_vs_B1_linear.png"), bbox_inches='tight', facecolor="w", dpi=300);

In [ ]:
fig.canvas.draw()

In [ ]:
plt.close(fig); del(fig, ax)

In [ ]:
fig, ax = plt.subplots(
    constrained_layout=True,
    figsize=(16*0.7,9*0.7),
)
ax.plot(
    amp/gauss,
    Mz_harmonic_amp_ratio,
    '-',
    color='gray',
    zorder=0,
)
ax.scatter(
    amp/gauss,
    Mz_harmonic_amp_ratio,
    color=color_list,
    edgecolors='black',
)
ax.set_title("$M_z$, " + title_prefix + ', nb_id = {}'.format(nb_id))
ax.set_xlabel('$B_1$ [G]')
ax.set_ylabel('ratio of 1st harmonic to fundamental [A.U.]');
ax.axvline(7.54, color='gray', linestyle='--', zorder=0, label='7.54 G')
ax.legend();

In [ ]:
fig.savefig(out_path(filename_prefix + "_Mz_harmonic_ratio_1_vs_B1.png"), bbox_inches='tight', facecolor="w", dpi=300);

In [ ]:
fig.canvas.draw()

In [ ]:
plt.close(fig); del(fig, ax)

In [ ]:
fig, ax = plt.subplots(
    constrained_layout=True,
    figsize=(16*0.7,9*0.7),
)
ax.plot(
    amp/gauss,
    Mz_harmonic0p5/Mz_harmonic0,
    '-',
    color='gray',
    zorder=0,
)
ax.scatter(
    amp/gauss,
    Mz_harmonic0p5/Mz_harmonic0,
    color=color_list,
    edgecolors='black',
)
ax.set_title(title_prefix + ', nb_id = {}'.format(nb_id))
ax.set_xlabel('$B_1$ [G]')
ax.set_ylabel('ratio of sideband 2 ({:.1f} MHZ) to fundamental ({:.1f} MHz) [A.U.]'.format(1.5*1425, 1425));
ax.axvline(7.54, color='gray', linestyle='--', zorder=0, label='7.54 G')
ax.legend();

In [ ]:
fig.savefig(out_path(filename_prefix + "Mz_0p5_harmonic_ratio_vs_B1.png"), bbox_inches='tight', facecolor="w", dpi=300);

In [ ]:
fig.canvas.draw()

In [ ]:
plt.close(fig); del(fig, ax)

# My

In [ ]:
fig, ax = plt.subplots(
    constrained_layout=True,
    figsize=(16*0.7,9*0.7),
)
for i, d in dat.items():
    if i in (0, len(dat)-1):
        this_label = "$B_1 = ${:.1f} G".format(d['amp']/gauss)
    else:
        this_label = None
    ax.plot(
        d['fft_freq_Hz'][i0:i_max]/GHz,
        d['ffty_avg'][i0:i_max],
        label=this_label,
#         label="{:.1f} G".format(d['bstat']/gauss),
        color=colors[i],
    )
# ax.plot(fft_freq_Hz[i0:i_max]/GHz, fft_avg.z[i0:i_max]);
ax.set_title("$M_y$, " + title_prefix + ', nb_id = {}'.format(nb_id))
ax.set_xlabel('frequency [GHz]')
ax.set_ylabel('FFT mag. ($M_y$/$M_s$), linear scaling');
ax.axvline(1425e6/GHz, color='gray', linestyle='--', zorder=0, label='1425 MHz')
ax.axvline(1.5*1425e6/GHz, color='black', linestyle=':', zorder=0, label='{:.1f} MHz'.format(1.5*1425))
# ax.set_yscale('log')
ax.legend();

In [ ]:
fig.savefig(out_path(filename_prefix + "_overlay_ffty_linear.png"), bbox_inches='tight', facecolor="w", dpi=300);

In [ ]:
fig.canvas.draw()

In [ ]:
plt.close(fig); del(fig, ax)

In [ ]:
fig, ax = plt.subplots(
    constrained_layout=True,
    figsize=(16*0.7,9*0.7),
)
for i, d in dat.items():
    if i in (0, len(dat)-1):
        this_label = "$B_1 = ${:.1f} G".format(d['amp']/gauss)
    else:
        this_label = None
    ax.plot(
        d['fft_freq_Hz'][i0:i_max]/GHz,
        d['ffty_avg'][i0:i_max],
        label=this_label,
        color=colors[i],
    )
ax.set_title("$M_y$, " + title_prefix + ', nb_id = {}'.format(nb_id))
ax.set_xlabel('frequency [GHz]')
ax.set_ylabel('FFT mag. ($M_y$/$M_s$), log scaling');
ax.axvline(1425e6/GHz, color='gray', linestyle='--', zorder=0, label='1425 MHz')
ax.axvline(1.5*1425e6/GHz, color='black', linestyle=':', zorder=0, label='{:.1f} MHz'.format(1.5*1425))
ax.set_yscale('log')
ax.legend();

In [ ]:
fig.savefig(out_path(filename_prefix + "_overlay_ffty_log.png"), bbox_inches='tight', facecolor="w", dpi=300);

In [ ]:
fig.canvas.draw()

In [ ]:
plt.close(fig); del(fig, ax)

In [ ]:
My_harmonic0_list = []
My_harmonic1_list = []
My_harmonic0p5_list = []
for i, d in dat.items():
    My_harmonic0_list.append(d['ffty_avg'][closest_index(d['fft_freq_Hz'], 1425.0e6)])
    My_harmonic1_list.append(d['ffty_avg'][closest_index(d['fft_freq_Hz'], 2*1425.0e6)])
    My_harmonic0p5_list.append(d['ffty_avg'][closest_index(d['fft_freq_Hz'], 1.5*1425.0e6)])
My_harmonic0 = np.array(My_harmonic0_list)
My_harmonic1 = np.array(My_harmonic1_list)
My_harmonic0p5 = np.array(My_harmonic0p5_list)
My_harmonic_amp_ratio = My_harmonic1/My_harmonic0

In [ ]:
My_harmonic_amp_ratio

In [ ]:
color_list = list(colors.values())

In [ ]:
fig, ax = plt.subplots(
    constrained_layout=True,
    figsize=(16*0.7,9*0.7),
)
ax.plot(
    amp/gauss,
    My_harmonic0,
    '.-',
    label='1425 MHz'
)
ax.plot(
    amp/gauss,
    My_harmonic1,
    '.-',
    label='2850 MHz',
)
ax.plot(
    amp/gauss,
    My_harmonic0p5,
    '.-',
    label='{:.1f} MHz'.format(1425*1.5),
)
ax.set_title("$M_y$, " + title_prefix + ', nb_id = {}'.format(nb_id))
ax.set_xlabel('$B_1$ [G]')
ax.set_ylabel('FFT mag. ($M_y$/$M_s$)');
ax.axvline(7.54, color='gray', linestyle='--', zorder=0, label='7.54 G')
ax.set_yscale('log')
ax.legend();

In [ ]:
fig.savefig(out_path(filename_prefix + "_My_harmonic_0_1_vs_B1_log.png"), bbox_inches='tight', facecolor="w", dpi=300);

In [ ]:
fig.canvas.draw()

In [ ]:
plt.close(fig); del(fig, ax)

In [ ]:
fig, ax = plt.subplots(
    constrained_layout=True,
    figsize=(16*0.7,9*0.7),
)
ax.plot(
    amp/gauss,
    My_harmonic0,
    '.-',
    label='1425 MHz'
)
ax.plot(
    amp/gauss,
    My_harmonic1,
    '.-',
    label='2850 MHz',
)
ax.plot(
    amp/gauss,
    My_harmonic0p5,
    '.-',
    label='{:.1f} MHz'.format(1425*1.5),
)
ax.set_title("$M_y$, " + title_prefix + ', nb_id = {}'.format(nb_id))
ax.set_xlabel('$B_1$ [G]')
ax.set_ylabel('FFT mag. ($M_y$/$M_s$)');
ax.axvline(7.54, color='gray', linestyle='--', zorder=0, label='7.54 G')
ax.legend();

In [ ]:
fig.savefig(out_path(filename_prefix + "_My_harmonic_0_1_vs_B1_linear.png"), bbox_inches='tight', facecolor="w", dpi=300);

In [ ]:
fig.canvas.draw()

In [ ]:
plt.close(fig); del(fig, ax)

In [ ]:
fig, ax = plt.subplots(
    constrained_layout=True,
    figsize=(16*0.7,9*0.7),
)
ax.plot(
    amp/gauss,
    My_harmonic_amp_ratio,
    '-',
    color='gray',
    zorder=0,
)
ax.scatter(
    amp/gauss,
    My_harmonic_amp_ratio,
    color=color_list,
    edgecolors='black',
)
ax.set_title("$M_y$, " + title_prefix + ', nb_id = {}'.format(nb_id))
ax.set_xlabel('$B_1$ [G]')
ax.set_ylabel('ratio of 1st harmonic to fundamental [A.U.]');
ax.axvline(7.54, color='gray', linestyle='--', zorder=0, label='7.54 G')
ax.legend();

In [ ]:
fig.savefig(out_path(filename_prefix + "_My_harmonic_ratio_1_vs_B1.png"), bbox_inches='tight', facecolor="w", dpi=300);

In [ ]:
fig.canvas.draw()

In [ ]:
plt.close(fig); del(fig, ax)

In [ ]:
fig, ax = plt.subplots(
    constrained_layout=True,
    figsize=(16*0.7,9*0.7),
)
ax.plot(
    amp/gauss,
    My_harmonic0p5/My_harmonic0,
    '-',
    color='gray',
    zorder=0,
)
ax.scatter(
    amp/gauss,
    My_harmonic0p5/My_harmonic0,
    color=color_list,
    edgecolors='black',
)
ax.set_title("$M_y$, " + title_prefix + ', nb_id = {}'.format(nb_id))
ax.set_xlabel('$B_1$ [G]')
ax.set_ylabel('ratio of sideband 2 ({:.1f} MHZ) to fundamental ({:.1f} MHz) [A.U.]'.format(1.5*1425, 1425));
ax.axvline(7.54, color='gray', linestyle='--', zorder=0, label='7.54 G')
ax.legend();

In [ ]:
fig.savefig(out_path(filename_prefix + "My_0p5_harmonic_ratio_vs_B1.png"), bbox_inches='tight', facecolor="w", dpi=300);

In [ ]:
fig.canvas.draw()

In [ ]:
plt.close(fig); del(fig, ax)

# Mx

In [ ]:
fig, ax = plt.subplots(
    constrained_layout=True,
    figsize=(16*0.7,9*0.7),
)
for i, d in dat.items():
    if i in (0, len(dat)-1):
        this_label = "$B_1 = ${:.1f} G".format(d['amp']/gauss)
    else:
        this_label = None
    ax.plot(
        d['fft_freq_Hz'][i0:i_max]/GHz,
        d['fftx_avg'][i0:i_max],
        label=this_label,
#         label="{:.1f} G".format(d['bstat']/gauss),
        color=colors[i],
    )
# ax.plot(fft_freq_Hz[i0:i_max]/GHz, fft_avg.z[i0:i_max]);
ax.set_title("$M_x$, " + title_prefix + ', nb_id = {}'.format(nb_id))
ax.set_xlabel('frequency [GHz]')
ax.set_ylabel('FFT mag. ($M_x$/$M_s$), linear scaling');
ax.axvline(1425e6/GHz, color='gray', linestyle='--', zorder=0, label='1425 MHz')
ax.axvline(1.5*1425e6/GHz, color='black', linestyle=':', zorder=0, label='{:.1f} MHz'.format(1.5*1425))
# ax.set_yscale('log')
ax.legend();

In [ ]:
fig.savefig(out_path(filename_prefix + "_overlay_fftx_linear.png"), bbox_inches='tight', facecolor="w", dpi=300);

In [ ]:
fig.canvas.draw()

In [ ]:
plt.close(fig); del(fig, ax)

In [ ]:
fig, ax = plt.subplots(
    constrained_layout=True,
    figsize=(16*0.7,9*0.7),
)
for i, d in dat.items():
    if i in (0, len(dat)-1):
        this_label = "$B_1 = ${:.1f} G".format(d['amp']/gauss)
    else:
        this_label = None
    ax.plot(
        d['fft_freq_Hz'][i0:i_max]/GHz,
        d['fftx_avg'][i0:i_max],
        label=this_label,
        color=colors[i],
    )
ax.set_title("$M_x$, " + title_prefix + ', nb_id = {}'.format(nb_id))
ax.set_xlabel('frequency [GHz]')
ax.set_ylabel('FFT mag. ($M_x$/$M_s$), log scaling');
ax.axvline(1425e6/GHz, color='gray', linestyle='--', zorder=0, label='1425 MHz')
ax.axvline(1.5*1425e6/GHz, color='black', linestyle=':', zorder=0, label='{:.1f} MHz'.format(1.5*1425))
ax.set_yscale('log')
ax.legend();

In [ ]:
fig.savefig(out_path(filename_prefix + "_overlay_fftx_log.png"), bbox_inches='tight', facecolor="w", dpi=300);

In [ ]:
fig.canvas.draw()

In [ ]:
plt.close(fig); del(fig, ax)

In [ ]:
Mx_harmonic0_list = []
Mx_harmonic1_list = []
Mx_harmonic0p5_list = []
for i, d in dat.items():
    Mx_harmonic0_list.append(d['fftx_avg'][closest_index(d['fft_freq_Hz'], 1425.0e6)])
    Mx_harmonic1_list.append(d['fftx_avg'][closest_index(d['fft_freq_Hz'], 2*1425.0e6)])
    Mx_harmonic0p5_list.append(d['fftx_avg'][closest_index(d['fft_freq_Hz'], 1.5*1425.0e6)])
Mx_harmonic0 = np.array(Mx_harmonic0_list)
Mx_harmonic1 = np.array(Mx_harmonic1_list)
Mx_harmonic0p5 = np.array(Mx_harmonic0p5_list)
Mx_harmonic_amp_ratio = Mx_harmonic1/Mx_harmonic0

In [ ]:
Mx_harmonic_amp_ratio

In [ ]:
color_list = list(colors.values())

In [ ]:
fig, ax = plt.subplots(
    constrained_layout=True,
    figsize=(16*0.7,9*0.7),
)
ax.plot(
    amp/gauss,
    Mx_harmonic0,
    '.-',
    label='1425 MHz'
)
ax.plot(
    amp/gauss,
    Mx_harmonic1,
    '.-',
    label='2850 MHz',
)
ax.plot(
    amp/gauss,
    Mx_harmonic0p5,
    '.-',
    label='{:.1f} MHz'.format(1425*1.5),
)
ax.set_title("$M_x$, " + title_prefix + ', nb_id = {}'.format(nb_id))
ax.set_xlabel('$B_1$ [G]')
ax.set_ylabel('FFT mag. ($M_x$/$M_s$)');
ax.axvline(7.54, color='gray', linestyle='--', zorder=0, label='7.54 G')
ax.set_yscale('log')
ax.legend();

In [ ]:
fig.savefig(out_path(filename_prefix + "_Mx_harmonic_0_1_vs_B1_log.png"), bbox_inches='tight', facecolor="w", dpi=300);

In [ ]:
fig.canvas.draw()

In [ ]:
plt.close(fig); del(fig, ax)

In [ ]:
fig, ax = plt.subplots(
    constrained_layout=True,
    figsize=(16*0.7,9*0.7),
)
ax.plot(
    amp/gauss,
    Mx_harmonic0,
    '.-',
    label='1425 MHz'
)
ax.plot(
    amp/gauss,
    Mx_harmonic1,
    '.-',
    label='2850 MHz',
)
ax.plot(
    amp/gauss,
    Mx_harmonic0p5,
    '.-',
    label='{:.1f} MHz'.format(1425*1.5),
)
ax.set_title("$M_x$, " + title_prefix + ', nb_id = {}'.format(nb_id))
ax.set_xlabel('$B_1$ [G]')
ax.set_ylabel('FFT mag. ($M_x$/$M_s$)');
ax.axvline(7.54, color='gray', linestyle='--', zorder=0, label='7.54 G')
ax.legend();

In [ ]:
fig.savefig(out_path(filename_prefix + "_Mx_harmonic_0_1_vs_B1_linear.png"), bbox_inches='tight', facecolor="w", dpi=300);

In [ ]:
fig.canvas.draw()

In [ ]:
plt.close(fig); del(fig, ax)

In [ ]:
fig, ax = plt.subplots(
    constrained_layout=True,
    figsize=(16*0.7,9*0.7),
)
ax.plot(
    amp/gauss,
    Mx_harmonic_amp_ratio,
    '-',
    color='gray',
    zorder=0,
)
ax.scatter(
    amp/gauss,
    Mx_harmonic_amp_ratio,
    color=color_list,
    edgecolors='black',
)
ax.set_title("$M_x$, " + title_prefix + ', nb_id = {}'.format(nb_id))
ax.set_xlabel('$B_1$ [G]')
ax.set_ylabel('ratio of 1st harmonic to fundamental [A.U.]');
ax.axvline(7.54, color='gray', linestyle='--', zorder=0, label='7.54 G')
ax.legend();

In [ ]:
fig.savefig(out_path(filename_prefix + "_Mx_harmonic_ratio_1_vs_B1.png"), bbox_inches='tight', facecolor="w", dpi=300);

In [ ]:
fig.canvas.draw()

In [ ]:
plt.close(fig); del(fig, ax)

In [ ]:
fig, ax = plt.subplots(
    constrained_layout=True,
    figsize=(16*0.7,9*0.7),
)
ax.plot(
    amp/gauss,
    Mx_harmonic0p5/Mx_harmonic0,
    '-',
    color='gray',
    zorder=0,
)
ax.scatter(
    amp/gauss,
    Mx_harmonic0p5/Mx_harmonic0,
    color=color_list,
    edgecolors='black',
)
ax.set_title("$M_x$, " + title_prefix + ', nb_id = {}'.format(nb_id))
ax.set_xlabel('$B_1$ [G]')
ax.set_ylabel('ratio of sideband 2 ({:.1f} MHZ) to fundamental ({:.1f} MHz) [A.U.]'.format(1.5*1425, 1425));
ax.axvline(7.54, color='gray', linestyle='--', zorder=0, label='7.54 G')
ax.legend();

In [ ]:
fig.savefig(out_path(filename_prefix + "Mx_0p5_harmonic_ratio_vs_B1.png"), bbox_inches='tight', facecolor="w", dpi=300);

In [ ]:
fig.canvas.draw()

In [ ]:
plt.close(fig); del(fig, ax)